In [1]:
import numpy as np
import itertools as itr

import os as os
import sys as sys 
import pandas as pd
import warnings
import time as time
import random

sys.path.append(os.path.join("..","Libraries","QML_lib"))
import Evo as evo
import DataBase 
import QMD
import QML
import ModelGeneration

import matplotlib.pyplot as plt

global paulis_list
paulis_list = {'i' : np.eye(2), 'x' : evo.sigmax(), 'y' : evo.sigmay(), 'z' : evo.sigmaz()}

warnings.filterwarnings("ignore", message='Negative weights occured', category=RuntimeWarning)

/usr/local/lib/python2.7/dist-packages/IPython/parallel.py:13: ShimWarning: The `IPython.parallel` package has been deprecated since IPython 4.0. You should import from ipyparallel instead.
  "You should import from ipyparallel instead.", ShimWarning)
/usr/local/lib/python2.7/dist-packages/qinfer/parallel.py:52: UserWarning: Could not import IPython parallel. Parallelization support will be disabled.
  "Could not import IPython parallel. "


In [52]:
class NewQMDClass():
    #TODO: rename as ModelsDevelopmentClass when finished
    def __init__(self, initial_op_list, true_operator='x', maxmodnum=3, gaussian=True):
#    def __init__(self, initial_op_list, true_op_list, true_param_list):

        trueOp = DataBase.operator(true_operator)
        self.InitialOpList = initial_op_list
        self.TrueOpList = trueOp.constituents_operators
        self.TrueParamsList = [random.random() for i in self.TrueOpList]
        #self.TrueHam = evo.getH(self.TrueParamsList, self.TrueOpList)
        self.TrueHam = np.tensordot(self.TrueParamsList, self.TrueOpList, axes=1)
        self.initiateDB()
        self.MaxModNum = maxmodnum #TODO: necessary?
        self.gaussian = gaussian
        
    def initiateDB(self):
        self.db, self.legacy_db, self.model_lists = \
            DataBase.launch_db(
                gen_list = self.InitialOpList,
                true_ops = self.TrueOpList,
                true_params = self.TrueParamsList
            )
    def addModel(self, model):
        DataBase.add_model(
            model_name = model,
            running_database = self.db,
            model_lists = self.model_lists,
            true_ops = self.TrueOpList,
            true_params = self.TrueParamsList
        )
            
            
    def getModelInstance(self, name):
        try: 
            instance = DataBase.get_qml_instance(self.db, name)
            return instance
        except: 
            if name in list(self.legacy_db['<Name>']):
                print("Operator in legacy databse - retired. ")
            else: 
                print("Model not found.")
    def getOperatorInstance(self, name):
        try: 
            return DataBase.get_operator_instance(self.db, name)
        except:
            if name in list(self.legacy_db['<Name>']):
                print("Operator in legacy databse - retired. ")
            else: 
                print("Operator not found.")
            
    def killModel(self, name):
        if name not in list(self.db['<Name>']):
            print("Cannot remove ", name, "; not in ", list(self.db["<Name>"]))
        else: 
            # Add to legacy_db
            DataBase.move_to_legacy(self.db, self.legacy_db, name)
            model_instance = self.getModelInstance(name)
            # Remove from self.db
            self.db = DataBase.remove_model(self.db, name)
            del model_instance
    
            #TODO: plot?
    #TODO: functionality required by QMD class: 
    # - champion rules
        

In [53]:
generators = ['x', 'xPy', 'yTz', 'x']
qmd= NewQMDClass(initial_op_list=generators, true_operator='x')

Initialization Ready
Initialization Ready
Initialization Ready
Initialization Ready
('Model ', 'x', ' not previously considered -- adding.')
Initialization Ready
('Model ', 'xPy', ' not previously considered -- adding.')
Initialization Ready
('Model ', 'yTz', ' not previously considered -- adding.')
Initialization Ready
('Model', 'x', ' previously considered at location', 0)


In [57]:
generators = ['x', 'xPy', 'yTz', 'x']
qmd= NewQMDClass(initial_op_list=generators, true_operator='yTz')

Initialization Ready
Initialization Ready
Initialization Ready
Initialization Ready
('Model ', 'x', ' not previously considered -- adding.')
Initialization Ready
('Model ', 'xPy', ' not previously considered -- adding.')
Initialization Ready
('Model ', 'yTz', ' not previously considered -- adding.')
Initialization Ready
('Model', 'x', ' previously considered at location', 0)


In [58]:
a = qmd.getModelInstance('yTz')

In [59]:
a.UpdateModel(1)

Initial time selected > 0.926621640589
Final time selected > 0.926621640589
Final Parameters mean and stdev:[ 0.4967481   0.57443083]


In [60]:
x = qmd.getModelInstance('x')
x.UpdateModel(1)

Initial time selected > 0.509242188824


ValueError: operands could not be broadcast together with shapes (4,4) (2,2) 

In [55]:
qmd.killModel('x')

('Model instance : ', <QML.ModelLearningClass instance at 0x7f5fbf0b8ef0>, ' for model ', 'x')


In [46]:
y = qmd.getModelInstance('x')

Model not found.


In [ ]:
name='x'
#DataBase.move_to_legacy(qmd_new.db, qmd_new.legacy_db, name)
#qmd.killModel(name)

n_exp = 20
for name in qmd.db['<Name>']:
    mod = qmd.getModelInstance(name)
    mod.UpdateModel(n_experiments=n_exp)
    

In [11]:
def run_iqle(model, qmd, n_exp):
    model_instance = qmd.getModelInstance(model)
    model_instance.UpdateModel(n_exp)
    #model_instance.BayesOnModelsWithinTrees
    kill_this_model = model_instance.testSurvival # return whether to kill this model or make it remain active but saturated, etc.
    if(kill_this_model):
        qmd.killModel(model)

In [12]:
run_iqle(model='x', qmd=qmd, n_exp=1)

Initial time selected > 1.12315309631
Final time selected > 1.12315309631
Final Parameters mean and stdev:[ 0.47501394  0.58493952]
('Model instance : ', <QML.ModelLearningClass instance at 0x7f5fbf44a248>, ' for model ', 'x')


In [13]:
qmd.legacy_db

,<Name>,Epoch_Finish,Epoch_Start,Param_Est_Final
0,x,10.0,0.0,"[[0.475013940894, 0.584939519547]]"
